# Python for Data Analysis v2 | Notes_ Chapter_10 数据聚合与分组运算

 本人以简书作者 SeanCheney 系列专题文章并结合原书为学习资源，记录个人笔记，仅作为知识记录及后期复习所用，原作者地址查看 [简书 SeanCheney](https://www.jianshu.com/u/130f76596b02)，如有错误，还望批评指教。——ZJ


>原作者：SeanCheney | [《利用Python进行数据分析·第2版》第10章 数据聚合与分组运算](https://www.jianshu.com/p/b94deb5c7eb1) | 來源：简书

>[Github:wesm](https://github.com/wesm/pydata-book) | [Github:中文 BrambleXu](https://github.com/BrambleXu/pydata-notebook)|
简书:[利用 Python 进行数据分析·第2版](https://www.jianshu.com/c/52882df3377a)

环境：Python 3.6 

---

# Chapter 10 数据聚合与分组运算

- 对数据集进行分组并对各组应用一个函数（无论是聚合还是转换），通常是数据分析工作中的重要环节。
- 在将数据集加载、融合、准备好之后，通常就是计算分组统计或生成透视表。
- pandas 提供了一个灵活高效的 gruopby 功能，它使你能以一种自然的方式对数据集进行切片、切块、摘要等操作。

关系型数据库和 SQL（Structured Query Language，结构化查询语言）能够如此流行的原因之一就是其能够方便地对数据进行连接、过滤、转换和聚合。

但是，像 SQL 这样的查询语言所能执行的分组运算的种类很有限。在本章中你将会看到，由于Python 和 pandas 强大的表达能力，我们可以执行复杂得多的分组运算（利用任何可以接受 pandas对象或 NumPy 数组的函数）。在本章中，你将会学到：

- 使用一个或多个键（形式可以是函数、数组或 DataFrame 列名）分割 pandas 对象。
- 计算分组的概述统计，比如数量、平均值或标准差，或是用户定义的函数。
- 应用组内转换或其他运算，如规格化、线性回归、排名或选取子集等。
- 计算透视表或交叉表。
- 执行分位数分析以及其它统计分组分析。

>笔记：对时间序列数据的聚合（groupby 的特殊用法之一）也称作重采样（resampling），本书将在第 11 章中单独对其进行讲解。

## 10.1 GroupBy机制

Hadley Wickham（许多热门 R 语言包的作者）创造了一个用于表示分组运算的术语"split-apply-combine"（拆分－应用－合并）。

- 第一个阶段，pandas 对象（无论是 Series、DataFrame 还是其他的）中的数据会根据你所提供的一个或多个键被拆分（split）为多组。拆分操作是在对象的特定轴上执行的。例如，**DataFrame 可以在其行（axis=0）或列（axis=1）上进行分组。**
- 然后，将一个函数应用（apply）到各个分组并产生一个新值。
- 最后，所有这些函数的执行结果会被合并（combine）到最终的结果对象中。结果对象的形式一般取决于数据上所执行的操作。图10-1大致说明了一个简单的分组聚合过程。

![](./images/10_1.png)

分组键可以有多种形式，且类型不必相同：

- 列表或数组，其长度与待分组的轴一样。
- 表示 DataFrame 某个列名的值。
- 字典或 Series，给出待分组轴上的值与分组名之间的对应关系。
- 函数，用于处理轴索引或索引中的各个标签。


注意，后三种都只是快捷方式而已，其最终目的仍然是产生一组用于拆分对象的值。如果觉得这些东西看起来很抽象，不用担心，我将在本章中给出大量有关于此的示例。首先来看看下面这个非常简单的表格型数据集（以 DataFrame 的形式）：

```
In [5]: df = pd.DataFrame({'key1':['a', 'a', 'b', 'b', 'a'],'key2':['on
   ...: e', 'two', 'one', 'two', 'one'],'data1':np.random.randn(5),'dat
   ...: a2':np.random.randn(5)})

In [6]: df
Out[6]:
      data1     data2 key1 key2
0  0.685031  0.037289    a  one
1 -0.947276 -0.721434    a  two
2  0.349532  0.082013    b  one
3  0.065207  0.549601    b  two
4  1.150530  0.602206    a  one


```

假设你想要按 key1 进行分组，并计算 data1 列的平均值。实现该功能的方式有很多，而我们这里要用的是：访问 data1，并根据 key1 调用 groupby：

```
In [7]: grouped = df['data1'].groupby(df['key1'])

In [8]: grouped
Out[8]: <pandas.core.groupby.SeriesGroupBy object at 0x000002F122B0EEF0>


```
变量 grouped 是一个 GroupBy 对象。它实际上还没有进行任何计算，只是含有一些有关分组键 `df['key1']`的中间数据而已。换句话说，该对象已经有了接下来对各分组执行运算所需的一切信息。例如，我们可以调用 GroupBy 的 mean 方法来计算分组平均值：


```
In [9]: grouped.mean()
Out[9]:
key1
a    0.296095
b    0.207369
Name: data1, dtype: float64

```
稍后我将详细讲解 `.mean()`的调用过程。这里最重要的是，数据（Series）根据分组键进行了聚合，产生了一个新的 Series，其索引为 key1 列中的唯一值。之所以结果中索引的名称为 key1，是因为原始 DataFrame 的列 `df['key1']`就叫这个名字。

如果我们一次传入多个数组的列表，就会得到不同的结果：

```
In [11]: means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [12]: means
Out[12]:
key1  key2
a     one     0.917780
      two    -0.947276
b     one     0.349532
      two     0.065207
Name: data1, dtype: float64

```

这里，我通过两个键对数据进行了分组，得到的 Series具有一个层次化索引（由唯一的键对组成）：

```
In [13]: means.unstack()
Out[13]:
key2       one       two
key1
a     0.917780 -0.947276
b     0.349532  0.065207

```

在这个例子中，分组键均为 Series。实际上，分组键可以是任何长度适当的数组：

```
In [14]: states = np.array(['Ohio', 'California', 'California', 'Ohio',
    ...:  'Ohio'])

In [15]: years = np.array([2005, 2005, 2006, 2005, 2006])

In [16]: df['data1'].groupby([states, years]).mean()
Out[16]:
California  2005   -0.947276
            2006    0.349532
Ohio        2005    0.375119
            2006    1.150530
Name: data1, dtype: float64

```
通常，分组信息就位于相同的要处理 DataFrame 中。这里，你还可以将列名（可以是字符串、数字或其他 Python 对象）用作分组键：


```
In [17]: df.groupby('key1').mean()
Out[17]:
         data1     data2
key1
a     0.296095 -0.027313
b     0.207369  0.315807

In [22]: df['data1'].groupby(df['key1']).sum()
Out[22]:
key1
a    0.888285
b    0.414739
Name: data1, dtype: float64

In [23]: 0.888285/3
Out[23]: 0.296095

In [18]: df.groupby(['key1','key2']).mean()
Out[18]:
              data1     data2
key1 key2
a    one   0.917780  0.319748
     two  -0.947276 -0.721434
b    one   0.349532  0.082013
     two   0.065207  0.549601

In [19]: df
Out[19]:
      data1     data2 key1 key2
0  0.685031  0.037289    a  one
1 -0.947276 -0.721434    a  two
2  0.349532  0.082013    b  one
3  0.065207  0.549601    b  two
4  1.150530  0.602206    a  one


```
你可能已经注意到了，第一个例子在执行`df.groupby('key1').mean()`时，结果中没有 key2 列。这是因为`df['key2']`不是数值数据（俗称“麻烦列”），所以被从结果中排除了。默认情况下，所有数值列都会被聚合，虽然有时可能会被过滤为一个子集，稍后就会碰到。

无论你准备拿`groupby`做什么，都有可能会用到`GroupBy`的`size`方法，它可以返回一个含有分组大小的 Series：

```
In [7]: df.groupby(['key1', 'key2']).size()
Out[7]:
key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

```
注意，任何分组关键词中的缺失值，都会被从结果中除去。

## 对分组进行迭代

GroupBy 对象支持迭代，可以产生一组二元元组（由分组名和数据块组成）。看下面的例子：

```
In [10]: for name, group in df.groupby('key1'):
    ...:     print('name:',name)
    ...:
    ...:     print('group:\n',group)
    ...:
    ...:
    ...:
name: a
group:
       data1     data2 key1 key2
0 -0.386072  1.489817    a  one
1 -0.257510  0.869976    a  two
4  1.061409  1.889561    a  one

name: b
group:
       data1     data2 key1 key2
2 -0.186201  0.317141    b  one
3 -0.349452  2.170885    b  two


```
对于多重键的情况，元组的第一个元素将会是由**键值组成的元组**：


```
In [12]: for (k1, k2), group in df.groupby(['key1', 'key2']):
    ...:     print((k1, k2))
    ...:     print(group,'\n')
    ...:
('a', 'one')
      data1     data2 key1 key2
0 -0.386072  1.489817    a  one
4  1.061409  1.889561    a  one

('a', 'two')
     data1     data2 key1 key2
1 -0.25751  0.869976    a  two

('b', 'one')
      data1     data2 key1 key2
2 -0.186201  0.317141    b  one

('b', 'two')
      data1     data2 key1 key2
3 -0.349452  2.170885    b  two


```

当然，你可以对这些数据片段做任何操作。有一个你可能会觉得有用的运算：将这些数据片段做成一个字典：

```
In [15]: pieces = dict(list(df.groupby('key1')))

In [16]: pieces['b']
Out[16]:
      data1     data2 key1 key2
2 -0.186201  0.317141    b  one
3 -0.349452  2.170885    b  two

In [17]: pieces
Out[17]:
{'a':       data1     data2 key1 key2
 0 -0.386072  1.489817    a  one
 1 -0.257510  0.869976    a  two
 4  1.061409  1.889561    a  one, 
 'b':       data1     data2 key1 key2
 2 -0.186201  0.317141    b  one
 3 -0.349452  2.170885    b  two}

```
`groupby` 默认是在 `axis=0` (横轴，行)上进行分组的，通过设置也可以在其他任何轴上进行分组。拿上面例子中的 df 来说，我们可以根据 `dtype` 对列进行分组：


```
In [18]: df.dtypes
Out[18]:
data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [19]: grouped = df.groupby(df.dtypes, axis=1)

In [20]: grouped
    ...:
Out[20]: <pandas.core.groupby.DataFrameGroupBy object at 0x000001ECF7148080>

```

可以如下打印分组：

```
In [21]: for dtype, group in grouped:
    ...:     print('dtype:', dtype)
    ...:     print('group:\n',group)
    ...:
dtype: float64
group:
       data1     data2
0 -0.386072  1.489817
1 -0.257510  0.869976
2 -0.186201  0.317141
3 -0.349452  2.170885
4  1.061409  1.889561

dtype: object
group:
   key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


```
## 选取一列或列的子集

对于由 DataFrame 产生的 GroupBy 对象，如果用一个（单个字符串）或一组（字符串数组）列名对其进行索引，就能实现选取部分列进行聚合的目的。也就是说：


```
In [22]: df.groupby('key1')['data1']

In [23]: df.groupby('key1')[['data2']]


```
是以下代码的语法糖：

```
In [24]: df['data1'].groupby(df['key1'])

In [25]: df[['data2']].groupby(df['key1'])

```
尤其对于大数据集，很可能只需要对部分列进行聚合。例如，在前面那个数据集中，如果只需计算 data2 列的平均值并以 DataFrame 形式得到结果，可以这样写：


```
In [26]: df.groupby(['key1', 'key2'])[['data2']].mean()
Out[26]:
              data2
key1 key2
a    one   1.689689 
     two   0.869976
b    one   0.317141
     two   2.170885

```
这种索引操作所返回的对象是一个已分组的 DataFrame（如果传入的是列表或数组）或已分组的 Series（如果传入的是标量形式的单个列名）：


```
In [27]: s_grouped = df.groupby(['key1', 'key2'])['data2']

In [28]: s_grouped
Out[28]: <pandas.core.groupby.SeriesGroupBy object at 0x000001ECF7178160>

In [29]: s_grouped.mean()
Out[29]:
key1  key2
a     one     1.689689
      two     0.869976
b     one     0.317141
      two     2.170885
Name: data2, dtype: float64

``` 
## 通过字典或 Series 进行分组

除数组以外，分组信息还可以其他形式存在。来看另一个示例 DataFrame：


```
In [30]: people = pd.DataFrame(np.random.randn(5,5), columns=['a',
In [30]: people = pd.DataFrame(np.random.randn(5,5), columns=['a'
    ...: ,'b','c','d','e'], index=['Joe', 'Steve', 'Wes', 'Jim',
    ...: 'Travis'])

In [31]: people.iloc[2:3, [1,2]] = np.nan

In [32]: people
Out[32]:
               a         b         c         d         e
Joe    -1.308416  1.849264 -1.742412  0.445570 -0.400641
Steve   0.401188 -0.430096  0.260777  0.077142  0.122350
Wes     0.160894       NaN       NaN -1.732852 -0.728693
Jim     0.134345 -0.817169  0.394712  1.479236  1.341673
Travis -0.269139  0.197604 -2.427321  0.412765 -0.759357


```
现在，假设已知列的分组关系，并希望根据分组计算列的和：

```
In [33]: mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
    ...:    ....:            'd': 'blue', 'e': 'red', 'f' : 'oran
    ...: ge'}
    ...:


```
现在，你可以将这个字典传给 groupby，来构造数组，但我们可以直接传递字典（我包含了键“f”来强调，存在未使用的分组键是可以的）：


```

In [34]: by_column = people.groupby(mapping, axis=1)

In [35]: by_column.sum() # 值相加
Out[35]:
            blue       red
Joe    -1.296842  0.140207
Steve   0.337919  0.093442
Wes    -1.732852 -0.567799
Jim     1.873948  0.658849
Travis -2.014557 -0.830892
```
Series 也有同样的功能，它可以被看做一个固定大小的映射：

```
In [37]: map_series = pd.Series(mapping)

In [38]: map_series
Out[38]:
a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [39]: people.groupby(map_series, axis=1).count()
Out[39]:
        blue  red
Joe        2    3
Steve      2    3
Wes        1    2
Jim        2    3
Travis     2    3


```
## 通过函数进行分组

- 比起使用字典或 Series，使用 Python 函数是一种更原生的方法定义分组映射。
- 任何被当做分组键的函数都会在各个索引值上被调用一次，其返回值就会被用作分组名称。

具体点说，以上一小节的示例 DataFrame 为例，其索引值为人的名字。你可以计算一个字符串长度的数组，更简单的方法是传入 len 函数：

```
In [40]: people.groupby(len).sum()
Out[40]:
          a         b         c         d         e
3 -1.013177  1.032095 -1.347700  0.191955  0.212339
5  0.401188 -0.430096  0.260777  0.077142  0.122350
6 -0.269139  0.197604 -2.427321  0.412765 -0.759357

```
将函数跟数组、列表、字典、Series 混合使用也不是问题，因为任何东西在内部都会被转换为数组：


```
In [43]: key_list = ['one', 'one', 'one', 'two', 'two']

In [44]: people.groupby([len, key_list]).min()
Out[44]:
              a         b         c         d         e
3 one -1.308416  1.849264 -1.742412 -1.732852 -0.728693
  two  0.134345 -0.817169  0.394712  1.479236  1.341673
5 one  0.401188 -0.430096  0.260777  0.077142  0.122350
6 two -0.269139  0.197604 -2.427321  0.412765 -0.759357

```
## 根据索引级别分组

层次化索引数据集最方便的地方就在于它能够根据轴索引的一个级别进行聚合：

```
In [48]: columns = pd.MultiIndex.from_arrays([['US', 'US', 'US',
    ...: 'JP', 'JP'], [1,3,5,1,3]],names=['cty', 'tenor'])

In [49]: hier_df= pd.DataFrame(np.random.randn(4,5), columns= col
    ...: umns)

In [50]: hier_df
Out[50]:
cty          US                            JP
tenor         1         3         5         1         3
0      1.049545  0.092344 -3.168629  1.484455 -0.545410
1     -0.338401 -0.573835 -0.086784 -0.794552  0.153434
2     -0.338139  0.083639 -1.901800  0.327133  0.607710
3     -2.315856 -0.567285 -1.425184 -1.590837  0.594141

```
要根据级别分组，使用 level 关键字传递级别序号或名字：

```
In [51]: hier_df.groupby(level='cty', axis=1).count()
Out[51]:
cty  JP  US
0     2   3
1     2   3
2     2   3
3     2   3

```

## 10.2 数据聚合

聚合指的是任何能够从数组产生标量值的数据转换过程。之前的例子已经用过一些，比如 mean、count、min 以及 sum 等。你可能想知道在 GroupBy 对象上调用 mean()时究竟发生了什么。许多常见的聚合运算（如表10-1所示）都有进行优化。然而，除了这些方法，你还可以使用其它的。

![](./images/10_1_b.png)

你可以使用自己发明的聚合运算，还可以调用分组对象上已经定义好的任何方法。例如，quantile 可以计算 Series 或 DataFrame 列的样本分位数。

虽然 quantile 并没有明确地实现于 GroupBy，但它是一个 Series 方法，所以这里是能用的。实际上，GroupBy 会高效地对 Series 进行切片，然后对各片调用piece.quantile(0.9)，最后将这些结果组装成最终结果：


```
In [52]: df
Out[52]:
      data1     data2 key1 key2
0 -0.386072  1.489817    a  one
1 -0.257510  0.869976    a  two
2 -0.186201  0.317141    b  one
3 -0.349452  2.170885    b  two
4  1.061409  1.889561    a  one

In [53]: grouped = df.groupby('key1')

In [54]: grouped['data1'].quantile(0.9)
Out[54]:
key1
a    0.797625
b   -0.202526
Name: data1, dtype: float64

In [55]: type(grouped)
Out[55]: pandas.core.groupby.DataFrameGroupBy


```

如果要使用你自己的聚合函数，只需将其传入 aggregate 或 agg 方法即可：

```
In [56]: def peak_to_peak(arr):
    ...:     return arr.max() - arr.min()
    ...:

In [57]: grouped.agg(peak_to_peak)
Out[57]:
         data1     data2
key1
a     1.447481  1.019585
b     0.163251  1.853744

```

你可能注意到注意，有些方法（如describe）也是可以用在这里的，即使严格来讲，它们并非聚合运算：


```
In [58]: grouped.describe()
Out[58]:
     data1
      \
     count      mean       std       min       25%       50%
 75%
key1

a      3.0  0.139275  0.801174 -0.386072 -0.321791 -0.257510  0.401949
b      2.0 -0.267826  0.115436 -0.349452 -0.308639 -0.267826 -0.227014

               data2
 \
           max count      mean       std       min       25%       50%
key1
a     1.061409   3.0  1.416451  0.513737  0.869976  1.179896  1.489817
b    -0.186201   2.0  1.244013  1.310795  0.317141  0.780577  1.244013


           75%       max
key1
a     1.689689  1.889561
b     1.707449  2.170885

```

在后面的10.3节，我将详细说明这到底是怎么回事。

>笔记：自定义聚合函数要比表10-1中那些经过优化的函数慢得多。这是因为在构造中间分组数据块时存在非常大的开销（函数调用、数据重排等）。

## 面向列的多函数应用

回到前面小费的例子。使用 read_csv 导入数据之后，我们添加了一个小费百分比的列 tip_pct：

```
In [59]:
In [59]: tips = pd.read_csv('examples/tips.csv')

In [60]: tips['tip_pct'] = tips['tip']/tips['total_bill']

In [61]: tips[:6]
Out[61]:
   total_bill   tip smoker  day    time  size   tip_pct
0       16.99  1.01     No  Sun  Dinner     2  0.059447
1       10.34  1.66     No  Sun  Dinner     3  0.160542
2       21.01  3.50     No  Sun  Dinner     3  0.166587
3       23.68  3.31     No  Sun  Dinner     2  0.139780
4       24.59  3.61     No  Sun  Dinner     4  0.146808
5       25.29  4.71     No  Sun  Dinner     4  0.186240


```
你已经看到，对 Series 或 DataFrame 列的聚合运算其实就是使用 aggregate（使用自定义函数）或调用诸如 mean、std 之类的方法。然而，你可能希望对不同的列使用不同的聚合函数，或一次应用多个函数。其实这也好办，我将通过一些示例来进行讲解。首先，我根据天和 smoker 对 tips 进行分组：

```
In [62]: grouped = tips.groupby(['day', 'smoker'])

```
注意，对于表10-1 中的那些描述统计，可以将函数名以字符串的形式传入：


```
In [63]: grouped_pct = grouped['tip_pct']

In [64]: grouped_pct.agg('mean')
Out[64]:
day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

```
如果传入一组函数或函数名，得到的 DataFrame 的列就会以相应的函数命名：

```
In [67]: grouped_pct.agg(['mean', 'std', peak_to_peak])
Out[67]:
                 mean       std  peak_to_peak
day  smoker
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

```

这里，我们传递了一组聚合函数进行聚合，独立对数据分组进行评估。

你并非一定要接受 GroupBy 自动给出的那些列名，特别是 lambda 函数，它们的名称是'<lambda>'，这样的辨识度就很低了（通过函数的 name 属性看看就知道了）。因此，如果传入的是一个由 (name,function)元组组成的列表，则各元组的第一个元素就会被用作 DataFrame 的列名（可以将这种二元元组列表看做一个有序映射）：


```

```
对于DataFrame，你还有更多选择，你可以定义一组应用于全部列的一组函数，或不同的列应用不同的函数。假设我们想要对tip_pct和total_bill列计算三个统计信息：

```

```
如你所见，结果 DataFrame 拥有层次化的列，这相当于分别对各列进行聚合，然后用concat将结果组装到一起，使用列名用作keys参数：

```

```
跟前面一样，这里也可以传入带有自定义名称的一组元组：

```

```
现在，假设你想要对一个列或不同的列应用不同的函数。具体的办法是向agg传入一个从列名映射到函数的字典：


```

```
只有将多个函数应用到至少一列时，DataFrame才会拥有层次化的列。

## 以“没有行索引”的形式返回聚合数据

到目前为止，所有示例中的聚合数据都有由唯一的分组键组成的索引（可能还是层次化的）。由于并不总是需要如此，所以你可以向groupby传入as_index=False以禁用该功能：



```

```
当然，对结果调用reset_index也能得到这种形式的结果。使用as_index=False方法可以避免一些不必要的计算。

## 10.3 apply：一般性的“拆分－应用－合并”

最通用的GroupBy方法是apply，本节剩余部分将重点讲解它。如图10-2所示，apply会将待处理的对象拆分成多个片段，然后对各片段调用传入的函数，最后尝试将各片段组合到一起。

![](./images/10_2.png)

回到之前那个小费数据集，假设你想要根据分组选出最高的5个tip_pct值。首先，编写一个选取指定列具有最大值的行的函数：

```

```
现在，如果对smoker分组并用该函数调用apply，就会得到：



```

```

这里发生了什么？top函数在DataFrame的各个片段上调用，然后结果由pandas.concat组装到一起，并以分组名称进行了标记。于是，最终结果就有了一个层次化索引，其内层索引值来自原DataFrame。

如果传给apply的函数能够接受其他参数或关键字，则可以将这些内容放在函数名后面一并传入：


```

```
笔记：除这些基本用法之外，能否充分发挥apply的威力很大程度上取决于你的创造力。传入的那个函数能做什么全由你说了算，它只需返回一个pandas对象或标量值即可。本章后续部分的示例主要用于讲解如何利用groupby解决各种各样的问题。

可能你已经想起来了，之前我在GroupBy对象上调用过describe：

```

```
在GroupBy中，当你调用诸如describe之类的方法时，实际上只是应用了下面两条代码的快捷方式而已：


```

```

## 禁止分组键

从上面的例子中可以看出，分组键会跟原始对象的索引共同构成结果对象中的层次化索引。将group_keys=False传入groupby即可禁止该效果：

```

```

## 分位数和桶分析

我曾在第8章中讲过，pandas有一些能根据指定面元或样本分位数将数据拆分成多块的工具（比如cut和qcut）。将这些函数跟groupby结合起来，就能非常轻松地实现对数据集的桶（bucket）或分位数（quantile）分析了。以下面这个简单的随机数据集为例，我们利用cut将其装入长度相等的桶中：


```

```
由cut返回的Categorical对象可直接传递到groupby。因此，我们可以像下面这样对data2列做一些统计计算：


```

```

这些都是长度相等的桶。要根据样本分位数得到大小相等的桶，使用qcut即可。传入labels=False即可只获取分位数的编号：

```

```
我们会在第12章详细讲解pandas的Categorical类型。

## 示例：用特定于分组的值填充缺失值

对于缺失数据的清理工作，有时你会用dropna将其替换掉，而有时则可能会希望用一个固定值或由数据集本身所衍生出来的值去填充NA值。这时就得使用fillna这个工具了。在下面这个例子中，我用平均值去填充NA值：




```

```
假设你需要对不同的分组填充不同的值。一种方法是将数据分组，并使用apply和一个能够对各数据块调用fillna的函数即可。下面是一些有关美国几个州的示例数据，这些州又被分为东部和西部：


```

```

['East'] * 4产生了一个列表，包括了['East']中元素的四个拷贝。将这些列表串联起来。

将一些值设为缺失：

```

```
我们可以用分组平均值去填充NA值:

```

```
外，也可以在代码中预定义各组的填充值。由于分组具有一个name属性，所以我们可以拿来用一下：


```

```
## 示例：随机采样和排列

假设你想要从一个大数据集中随机抽取（进行替换或不替换）样本以进行蒙特卡罗模拟（Monte Carlo simulation）或其他分析工作。“抽取”的方式有很多，这里使用的方法是对Series使用sample方法：


```

```

现在我有了一个长度为52的Series，其索引包括牌名，值则是21点或其他游戏中用于计分的点数（为了简单起见，我当A的点数为1）：

```

```
现在，根据我上面所讲的，从整副牌中抽出5张，代码如下：


```

```
假设你想要从每种花色中随机抽取两张牌。由于花色是牌名的最后一个字符，所以我们可以据此进行分组，并使用apply：


```

```

或者，也可以这样写：

```

```
## 示例：分组加权平均数和相关系数

根据groupby的“拆分－应用－合并”范式，可以进行DataFrame的列与列之间或两个Series之间的运算（比如分组加权平均）。以下面这个数据集为例，它含有分组键、值以及一些权重值：


```

```

然后可以利用category计算分组加权平均数：

```

```
另一个例子，考虑一个来自Yahoo!Finance的数据集，其中含有几只股票和标准普尔500指数（符号SPX）的收盘价：


```

```
来做一个比较有趣的任务：计算一个由日收益率（通过百分数变化计算）与SPX之间的年度相关系数组成的DataFrame。下面是一个实现办法，我们先创建一个函数，用它计算每列和SPX列的成对相关系数：


```

```
接下来，我们使用pct_change计算close_px的百分比变化：


```

```
最后，我们用年对百分比变化进行分组，可以用一个一行的函数，从每行的标签返回每个datetime标签的year属性：


```

```

当然，你还可以计算列与列之间的相关系数。这里，我们计算Apple和Microsoft的年相关系数：

```

```
## 示例：组级别的线性回归

顺着上一个例子继续，你可以用groupby执行更为复杂的分组统计分析，只要函数返回的是pandas对象或标量值即可。例如，我可以定义下面这个regress函数（利用statsmodels计量经济学库）对各数据块执行普通最小二乘法（Ordinary Least Squares，OLS）回归：



```

```

现在，为了按年计算AAPL对SPX收益率的线性回归，执行：

```

```
## 10.4 透视表和交叉表

透视表（pivot table）是各种电子表格程序和其他数据分析软件中一种常见的数据汇总工具。它根据一个或多个键对数据进行聚合，并根据行和列上的分组键将数据分配到各个矩形区域中。在Python和pandas中，可以通过本章所介绍的groupby功能以及（能够利用层次化索引的）重塑运算制作透视表。DataFrame有一个pivot_table方法，此外还有一个顶级的pandas.pivot_table函数。除能为groupby提供便利之外，pivot_table还可以添加分项小计，也叫做margins。

回到小费数据集，假设我想要根据day和smoker计算分组平均数（pivot_table的默认聚合类型），并将day和smoker放到行上：


```

```

可以用groupby直接来做。现在，假设我们只想聚合tip_pct和size，而且想根据time进行分组。我将smoker放到列上，把day放到行上：

```

```

还可以对这个表作进一步的处理，传入margins=True添加分项小计。这将会添加标签为All的行和列，其值对应于单个等级中所有数据的分组统计：

```

```
这里，All值为平均数：不单独考虑烟民与非烟民（All列），不单独考虑行分组两个级别中的任何单项（All行）。

要使用其他的聚合函数，将其传给aggfunc即可。例如，使用count或len可以得到有关分组大小的交叉表（计数或频率）：

```

```
如果存在空的组合（也就是NA），你可能会希望设置一个fill_value：

```

```
pivot_table的参数说明请参见表10-2。

![](./images/10_2_b.png)

## 交叉表：crosstab

交叉表（cross-tabulation，简称crosstab）是一种用于计算分组频率的特殊透视表。看下面的例子：

```

```
作为调查分析的一部分，我们可能想要根据国籍和用手习惯对这段数据进行统计汇总。虽然可以用pivot_table实现该功能，但是pandas.crosstab函数会更方便：

```

```

crosstab的前两个参数可以是数组或Series，或是数组列表。就像小费数据：

```

```
## 10.5 总结

掌握pandas数据分组工具既有助于数据清理，也有助于建模或统计分析工作。在第14章，我们会看几个例子，对真实数据使用groupby。

在下一章，我们将关注时间序列数据。

```

```
